# C#

## Just iterate to goal, recusively

In [ ]:
static ulong Go(ulong target, ulong x) {
    if (x != target)
        return Go(target, x + 1);
    else return x;
}
Go(10,0)

## Calculate the sum of values in a list,
Using a recursive algorithm. Unusual in C#, but common in other languages! 

In [ ]:
static ulong SumRec(ulong[] l) => l switch {
    [] => 0,
    [var x, .. var xs] => x + SumRec(xs)
};
SumRec(new ulong[] { 2, 3, 6, 8 })

## Using sequences instead of arrays with the pattern matching syntax

In [ ]:
static ulong SumRecSeq(IEnumerable<ulong> s) {
    // using long syntax for clarity
    if (s.Any()) {
        return s.First() + SumRecSeq(s.Skip(1));
    }
    else {
        return 0;
    }
}
SumRec(new ulong[] { 2, 3, 6, 8 })

## Passing result along as a parameter and the calculation is performed *before* the call.
It is no longer necessary to return to the same call instance later to complete the calculation.

In [ ]:
static ulong SumRecSeqTail(IEnumerable<ulong> s, ulong result = 0) {
  // using long syntax for clarity
  if (s.Any()) {
    return SumRecSeqTail(s.Skip(1), result + s.First());
  }
  else {
    return result;
  }
}
SumRecSeqTail(new ulong[] { 2, 3, 6, 8 })

## Prevent any performance issues due to the list handling
which is a bit contrived in the examples above, especially for C#), here is a variation of the last function that still calculates a sum in a very similar way, but does not use a list.

In [ ]:
static ulong SumRecMaxTail(ulong max, ulong val, ulong result = 0) {
  if (val < max)
    return SumRecMaxTail(max, val + 1, result + val);
  else
    return result;
}
SumRecMaxTail(20, 0)

## Trampolining

In [ ]:

  // Encapsulation of the either/or result 
  // Not needed in dynamic languages
  public class TrampolineResult<Tout> {
    public TrampolineResult(Func<TrampolineResult<Tout>> continuation) {
      IsContinuation = true;
      this.Continuation = continuation;
      this.Result = default!; // satisfy compiler
    }
    public bool IsContinuation { get; }
    public Func<TrampolineResult<Tout>> Continuation { get; }

    public TrampolineResult(Tout result) {
      IsContinuation = false;
      this.Result = result;
      this.Continuation = default!; // satisfy compiler
    }
    public Tout Result { get; }
  }

  // Syntactic convenience -- maybe OOP people like this syntax?
  public class TrampolineResult {
    public static TrampolineResult<Tout> From<Tout>(Tout result) => new(result);
    public static TrampolineResult<Tout> From<Tout>(Func<TrampolineResult<Tout>> continuation) => new(continuation);
  }

  // Or maybe like this for a more FP feel?
  static TrampolineResult<Tout> Tr<Tout>(Tout result) => new(result);
  static TrampolineResult<Tout> Tr<Tout>(Func<TrampolineResult<Tout>> continuation) => new(continuation);

  public static Tout Trampoline<Tout>(Func<TrampolineResult<Tout>> f) {
    var currentResult = new TrampolineResult<Tout>(f);
    while (currentResult.IsContinuation)
      currentResult = currentResult.Continuation.Invoke();
    return currentResult.Result;
  }

  static TrampolineResult<ulong> SumTrampoline(ulong x, ulong current = 0) {
    if (x == 0)
      return Tr(current);
    else
      return Tr(() => SumTrampoline(x - 1, current + x));
  }

In [ ]:
Trampoline(() => SumTrampoline(100))

In [ ]:
Trampoline(() => SumTrampoline(300000))

## CPS

In [ ]:
static void SumCps(ulong[] l, Action<ulong> continuation) {
  switch (l) {
    case []: continuation(0); break;
    case [var x, .. var xs]: SumCps(xs, ix => continuation(ix + x)); break;
  };
}
static void SumCpsSeq(IEnumerable<ulong> s, Action<ulong> continuation) {
  if (!s.Any()) {
    continuation(0);
    return;
  }
  SumCpsSeq(s.Skip(1), ix => continuation(ix + s.First()));
}
static Action<ulong> OutputResult(string prefix) => (ulong value) => Console.WriteLine($"{prefix}: {value}");

In [ ]:
SumCps(new ulong[] { 2, 3, 6, 8 }, OutputResult("SumCps"));

In [ ]:
SumCpsSeq(new ulong[] { 2, 3, 6, 8 }, OutputResult("SumCpsSeq"));